#### Melhorias
1) melhorar a funcao get_similar_documents

### Pipeline MCP
1) Implementar código para conexão com o banco de dados Chroma DB e MySQL
2) Criar agente de tomada de decisão que verifica se é necessário utilizar o rag para extração de conhecimento dos documentos
3) Construção do agente responsável por gerar uma resposta deopis de obter os documentos relevantes com RAG
4) Juntar tudo
5) Desenvolver um frontend (?)

##### Tarefa 1: Conexão com o ChromaDB

###### Objetivo: Armazenar informações para tratamento de doenças do coração

In [1]:
import pandas as pd
import chromadb
from faker import Faker
import fitz
import sqlite3
from typing import List
from sentence_transformers import SentenceTransformer, util
from ollama import Client
import json

model = SentenceTransformer('all-MiniLM-L6-v2')
model_serafim = SentenceTransformer('PORTULAN/serafim-900m-portuguese-pt-sentence-encoder')

d:\usuarios\pedro.watanabe\Meus Documentos\tcc-pedro\agents\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def connect_chromadb_remote():
  """
  Faz conexão via API com o ChromaDB e utilizand o banco de dados na Cloud
  """
  client = chromadb.CloudClient(
    api_key='ck-E6LFXqSmT2dZEoVLmyBt27qQtbZ1pHwzLCiFbe7Z38pm',
    tenant='cfd9bab2-f948-4809-b5c8-ab104d1b8d5c',
    database='test'
  )
  return client

def connect_chromadb_local(path):
  """
  Cria um banco de dados local
  """
  client = chromadb.PersistentClient(path=path)
  return client

## Código para criar o banco de dados e a collection
chromadb_client = chromadb.PersistentClient(path="./incor_db")
collection = chromadb_client.get_or_create_collection(name="informacoes_doencas_cardiovasculares", metadata={"hnsw:space": "cosine"})

In [3]:
def filtering_documents_by_threshold(documents: list, distances: list, similarity_threshold: float):
  filtered_documents = []
  filtered_distances = []

  for document, distance in zip(documents[0], distances[0]):
      if distance <= similarity_threshold:
          filtered_documents.append(document)
          filtered_distances.append(distance)

  filtered_results = {
    "documents": filtered_documents,
    "distances": filtered_distances
  }

  return filtered_results


def get_similar_documents(query_text: str, n_results: int, similarity_threshold: float) -> list:
  """
  A métrica de distância utilizada é o l2, onde quanto menor o resultado mais similares são os textos. 
  """
  
  results = collection.query(
    query_texts=[query_text],
    n_results=n_results
  )

  filtered_results = filtering_documents_by_threshold(documents=results['documents'], distances=results['distances'], similarity_threshold=similarity_threshold)

  return filtered_results

In [4]:
results = collection.query(
query_texts=["As doenças cardiovasculares são as principais causas de morte no mundo , sendo responsáveis"],
n_results=2
)

filtered_results = filtering_documents_by_threshold(documents=results['documents'], distances=results['distances'], similarity_threshold=0.5)

filtered_results

{'documents': ['O VO2pico obtido através da equação preditora elaborada \nneste estudo com base no desempenho no TUG demonstrou \nconcordância com o VO2pico determinado pelo TCPE na \nmesma amostra, demonstrando ser um método adequado \npara estimar a ACR de cardiopatas. Em uma metanálise \ncom adultos saudáveis, Kodama et al.12 sugeriram que a \nACR seria um importante preditor de mortalidade e eventos \ncardiovasculares. Apesar da sua amostra apresentar diferentes \ncaracterísticas do nosso estudo, é possível inferir que uma \nmelhor ACR está associada a menores riscos de complicações \ncardiovasculares. É importante frisar que na literatura ainda são \nescassos os estudos que relacionem o teste TUG à população \nde pacientes cardiopatas A ACR estabelecida pelo VO2pico é um importante \ncomponente de avaliação da saúde, pois, de acordo com \nCarvalho et al.3 e Ritt et al.,5 trata-se de um determinante \nque deve ser mensurado periodicamente em pacientes \ncardiopatas, com o intuito d

In [5]:
def semantic_chunker(text: str, model: SentenceTransformer, similarity_threshold: float = 0.85) -> List[str]:
    sentences = text.split('.\n')

    embeddings = model.encode(sentences, convert_to_tensor=True)

    chunks = []
    current_chunk = [sentences[0]]

    for i in range(1, len(sentences)):
        similarity = util.pytorch_cos_sim(embeddings[i - 1], embeddings[i]).item()
        if similarity > similarity_threshold:
            current_chunk.append(sentences[i])
        else:
            chunks.append(' '.join(current_chunk))
            current_chunk = [sentences[i]]

    if current_chunk:
        chunks.append(' '.join(current_chunk))

    return chunks

In [6]:
import fitz

def extrair_texto_de_pdf_pymupdf(caminho_pdf):
    """
    Extrai o texto de todas as páginas de um arquivo PDF usando PyMuPDF.
    """
    texto_completo = ""
    try:
        documento = fitz.open(caminho_pdf)

        for pagina_num in range(len(documento)):
            pagina = documento.load_page(pagina_num)
            texto_completo += pagina.get_text()

        documento.close()

    except Exception as e:
        print(f"Ocorreu um erro: {e}")
        return None

    return texto_completo

caminho_do_meu_pdf = "./data/papers_pdf/teste_artigo.pdf"
texto_extraido = extrair_texto_de_pdf_pymupdf(caminho_do_meu_pdf)

In [7]:
def removing_text_with_less_than_n_char(texts: str, n: int):
    """Função que removerá textos que contém menos que 50 caracteres. 
        Objetivo: Prevenir que seja imputado textos pequenos no modelo de IA. É provável que esses textos pequenos são alguma falha.
    """
    for i, text in enumerate(texts):
        if len(text) <= n:
            texts.pop(i)
    
    return texts

def preprocessing_texts(extracted_text: str, model: SentenceTransformer, similarity_threshold: float):
    chunks = semantic_chunker(text=texto_extraido, model=model, similarity_threshold=0.6)
    new_chunks = removing_text_with_less_than_n_char(texts=chunks, n=100)

    return new_chunks

In [8]:
chunks = semantic_chunker(text=texto_extraido, model=model, similarity_threshold=0.75)
new_chunks = removing_text_with_less_than_n_char(texts=chunks, n=100)

ids = [f"id{i}" for i in range(len(chunks))]

collection.add(
    ids=ids,
    documents=chunks
)

In [9]:
total_de_documentos = collection.count()
total_de_documentos

69

In [10]:
def calcular_tamanho_medio_dos_documentos(documentos):
    tamanho_documentos = 0
    for documento in documentos:
        tamanho_documentos += len(documento)
    
    return tamanho_documentos/len(documentos)

In [11]:
from IPython.display import display, Markdown

query_text = "As doenças cardiovasculares causas de morte"
n_results = 5

sentences = get_similar_documents(query_text=query_text, n_results=n_results, similarity_threshold=0.5)
len(sentences['documents'])

2

In [12]:
for distance, document in zip(sentences['distances'], sentences['documents']):
    print("SIMILARIDADE: ", distance)
    print(document)
    print("="*30)


SIMILARIDADE:  0.4436817765235901
O VO2pico obtido através da equação preditora elaborada 
neste estudo com base no desempenho no TUG demonstrou 
concordância com o VO2pico determinado pelo TCPE na 
mesma amostra, demonstrando ser um método adequado 
para estimar a ACR de cardiopatas. Em uma metanálise 
com adultos saudáveis, Kodama et al.12 sugeriram que a 
ACR seria um importante preditor de mortalidade e eventos 
cardiovasculares. Apesar da sua amostra apresentar diferentes 
características do nosso estudo, é possível inferir que uma 
melhor ACR está associada a menores riscos de complicações 
cardiovasculares. É importante frisar que na literatura ainda são 
escassos os estudos que relacionem o teste TUG à população 
de pacientes cardiopatas A ACR estabelecida pelo VO2pico é um importante 
componente de avaliação da saúde, pois, de acordo com 
Carvalho et al.3 e Ritt et al.,5 trata-se de um determinante 
que deve ser mensurado periodicamente em pacientes 
cardiopatas, com o intuito

##### Tarefa 1.2: Conexão com o Banco de Dados Relacional (MySQL)

###### Objetivo: Será utilizada para obter informações dos pacientes

In [13]:
import sqlite3
from faker import Faker
import random

In [14]:
# Criação dos dados sintéticos
mysql_connection = sqlite3.connect('pacientes.db')

cursor = mysql_connection.cursor()

mysql_connection.row_factory = sqlite3.Row

cursor.execute('''
CREATE TABLE IF NOT EXISTS pacientes (
    paciente_id INTEGER PRIMARY KEY,
    nome_completo VARCHAR(255) NOT NULL,
    data_nascimento DATE NOT NULL,
    cpf VARCHAR(14) NOT NULL UNIQUE,
    genero VARCHAR(50),
    tipo_sanguineo VARCHAR(5),
    endereco TEXT,
    telefone VARCHAR(20),
    email VARCHAR(100),
    nome_convenio VARCHAR(100),
    numero_carteirinha VARCHAR(50),
    alergias TEXT,
    doencas_cronicas TEXT,
    medicamentos_em_uso TEXT,
    historico_cirurgico TEXT,
    contato_emergencia_nome VARCHAR(255),
    contato_emergencia_telefone VARCHAR(20),
    data_cadastro TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
''')

mysql_connection.commit()

In [15]:
tipos_sanguineos = ['A+', 'A-', 'B+', 'B-', 'AB+', 'AB-', 'O+', 'O-']
alergias_comuns = ['Penicilina', 'Ibuprofeno', 'Aspirina', 'Frutos do mar', 'Amendoim', 'Látex', 'Pólen']
doencas_cronicas_comuns = ['Hipertensão', 'Diabetes Tipo 2', 'Asma', 'Artrite Reumatoide', 'Hipotiroidismo']
medicamentos_comuns = ['Losartana', 'Metformina', 'Salbutamol', 'Prednisona', 'Levotiroxina', 'Sinvastatina']
cirurgias_comuns = ['Apendicectomia', 'Cesariana', 'Remoção da vesícula biliar', 'Artroplastia de joelho']

fake = Faker('pt_BR')

for _ in range(50):
    paciente_id = fake.random_number(digits=15, fix_len=False)
    nome_completo = fake.name()
    data_nascimento = fake.date_of_birth(minimum_age=10, maximum_age=90)
    cpf = fake.cpf()
    genero = random.choice(['Masculino', 'Feminino', 'Outro'])
    tipo_sanguineo = random.choice(tipos_sanguineos)
    endereco = fake.address()
    telefone = fake.phone_number()
    email = fake.email()
    nome_convenio = random.choice(['SulAmérica', 'Bradesco Saúde', 'Amil', 'Unimed', 'NotreDame Intermédica'])
    numero_carteirinha = fake.random_number(digits=15, fix_len=True)

    # Gerando listas de informações médicas (podem ser vazias)
    alergias = ', '.join(random.sample(alergias_comuns, k=random.randint(0, 2))) or 'Nenhuma conhecida'
    doencas_cronicas = ', '.join(random.sample(doencas_cronicas_comuns, k=random.randint(0, 2))) or 'Nenhuma conhecida'
    medicamentos_em_uso = ', '.join(random.sample(medicamentos_comuns, k=random.randint(0, 3))) or 'Nenhum'
    historico_cirurgico = ', '.join(random.sample(cirurgias_comuns, k=random.randint(0, 1))) or 'Nenhum'

    contato_emergencia_nome = fake.name()
    contato_emergencia_telefone = fake.phone_number()

    sql_insert = """
    INSERT INTO pacientes (
        paciente_id, nome_completo, data_nascimento, cpf, genero, tipo_sanguineo, endereco,
        telefone, email, nome_convenio, numero_carteirinha, alergias, doencas_cronicas,
        medicamentos_em_uso, historico_cirurgico, contato_emergencia_nome,
        contato_emergencia_telefone
    ) VALUES (
        ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?
    )
    """

    dados_paciente = (
      paciente_id, nome_completo, data_nascimento, cpf, genero, tipo_sanguineo, endereco,
      telefone, email, nome_convenio, numero_carteirinha, alergias, doencas_cronicas,
      medicamentos_em_uso, historico_cirurgico, contato_emergencia_nome,
      contato_emergencia_telefone
    )

    cursor.execute(sql_insert, dados_paciente)

mysql_connection.commit()

print("Pacientes cadastrados:")
cursor.execute("SELECT * FROM pacientes")
for paciente in cursor.fetchall():
    print({paciente})

mysql_connection.close()

C:\Users\pedro.watanabe\AppData\Local\Temp\ipykernel_13512\1183390171.py:49: DeprecationWarning: The default date adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  cursor.execute(sql_insert, dados_paciente)


Pacientes cadastrados:
{(2709738358990, 'Dom Pimenta', '2006-05-04', '246.158.379-09', 'Feminino', 'B-', 'Colônia Francisco Leão, 512\nVila Real 1ª Seção\n17091-064 Montenegro da Serra / PB', '+55 81 2429-2631', 'pachecolucas-gabriel@example.com', 'Unimed', '104534048270595', 'Nenhuma conhecida', 'Asma, Hipotiroidismo', 'Nenhum', 'Nenhum', 'Bento Ribeiro', '(084) 9056 9676', '2025-10-02 12:39:45')}
{(2953168225046, 'Ísis Almeida', '1959-08-08', '470.561.293-07', 'Masculino', 'O+', 'Trevo Gabriela da Costa, 269\nPirineus\n01679-153 Caldeira / SE', '(021) 9450-1079', 'nicoleporto@example.com', 'Bradesco Saúde', '660552342421074', 'Penicilina, Pólen', 'Hipertensão, Diabetes Tipo 2', 'Nenhum', 'Cesariana', 'Gustavo Abreu', '61 6308-7670', '2025-09-29 12:56:37')}
{(9001165133357, 'Helena Cassiano', '1943-07-28', '957.236.804-47', 'Outro', 'O-', 'Rua Enrico da Paz, 47\nSão Luiz\n96134108 Martins / RJ', '(081) 3245 1497', 'xgomes@example.org', 'SulAmérica', '428681310344307', 'Amendoim', 'Art

In [16]:
### SCRIPT para procurar paciente pelo paciente_id

mysql_connection = sqlite3.connect('pacientes.db')
cursor = mysql_connection.cursor()
mysql_connection.row_factory = sqlite3.Row

## Substituir o código abaixo por um único valor para paicente_id
cursor.execute("SELECT * FROM pacientes")
paciente_id = cursor.fetchone()[0]
#################


# Uma das ferramentas para o agente de IA
def get_patient_by_id(patient_id: int):
  mysql_connection = sqlite3.connect('pacientes.db')
  mysql_connection.row_factory = sqlite3.Row
  cursor = mysql_connection.cursor()

  query_patient_data = (f'''
  SELECT * FROM pacientes
  WHERE paciente_id = ?
  ''')

  cursor.execute(query_patient_data, (patient_id, ))
  patient_data = cursor.fetchone()

  mysql_connection.close()
  
  return dict(patient_data)

patient_data = get_patient_by_id(paciente_id)
print(patient_data)

mysql_connection.close()

{'paciente_id': 2709738358990, 'nome_completo': 'Dom Pimenta', 'data_nascimento': '2006-05-04', 'cpf': '246.158.379-09', 'genero': 'Feminino', 'tipo_sanguineo': 'B-', 'endereco': 'Colônia Francisco Leão, 512\nVila Real 1ª Seção\n17091-064 Montenegro da Serra / PB', 'telefone': '+55 81 2429-2631', 'email': 'pachecolucas-gabriel@example.com', 'nome_convenio': 'Unimed', 'numero_carteirinha': '104534048270595', 'alergias': 'Nenhuma conhecida', 'doencas_cronicas': 'Asma, Hipotiroidismo', 'medicamentos_em_uso': 'Nenhum', 'historico_cirurgico': 'Nenhum', 'contato_emergencia_nome': 'Bento Ribeiro', 'contato_emergencia_telefone': '(084) 9056 9676', 'data_cadastro': '2025-10-02 12:39:45'}


##### Tarefa 2: Criação do agente capaz de tomar decisões

In [17]:
# Passo 1
## Conexão com o servidor local

# Passo 2
## Criação do prompt com os critérios e regras que o agente utilizará para tomar a decisão
    # Pergunta:
        # Quais serão os critéiros e regras que serão utilizados para decisão?

# Passo 3
## Decidir um schema de saída
    ### Talvez: {"tools": function}

#### Próximos passos:
1.   Estudar sobre os metadados do chromadb. É possível realizar uma busca dos documentos a partir dos metadados?
2.   Definir ferramentas que será possível utilizar




In [18]:
# Ferramentas
def get_similar_documents(query_text: str, n_documents: int, similarity_threshold: float = 0.5) -> list:
  results = collection.query(
    query_texts=[query_text],
    n_results=n_documents
  )

  filtered_results = filtering_documents_by_threshold(documents=results['documents'], distances=results['distances'], similarity_threshold=similarity_threshold)

  return filtered_results


def get_patient_by_id(patient_id: int):
  mysql_connection = sqlite3.connect('pacientes.db')
  cursor = mysql_connection.cursor()

  query_patient_data = (f'''
  SELECT * FROM pacientes
  WHERE paciente_id = ?
  ''')

  cursor.execute(query_patient_data, (paciente_id, ))
  patient_data = cursor.fetchone()

  mysql_connection.close()

  return patient_data


In [19]:
router_system_prompt = """# **PERSONA**
Você é um agente roteador de ferramentas altamente preciso e especializado no domínio da saúde. Você não responde diretamente ao usuário. Sua única função é analisar a solicitação do usuário e determinar qual, se alguma, das ferramentas disponíveis é a mais apropriada para a tarefa.

# **OBJETIVO PRINCIPAL**
Analisar a pergunta do usuário (`user_input`) e retornar um objeto JSON que representa a chamada de ferramenta mais adequada para resolver a solicitação. Se nenhuma ferramenta for adequada, você deve retornar um JSON específico indicando isso.

# **DESCRIÇÃO DAS FERRAMENTAS DISPONÍVEIS**

**Ferramenta 1: `get_similar_documents`**
* **Descrição:** Realiza uma busca semântica em uma base de conhecimento sobre doenças cardiológicas. É ideal para encontrar informações, artigos ou documentos relacionados a condições, tratamentos, sintomas e diagnósticos.
* **Quando Usar:** Utilize esta função quando a pergunta do usuário for de natureza informativa ou teórica sobre cardiologia. Por exemplo: "O que é insuficiência cardíaca?", "Quais os tratamentos para arritmia?", "sintomas de infarto".
* **Argumentos:**
    * `main_topic` (string): O tópico central da pergunta do usuário, extraído e resumido. Deve ser conciso e direto.

**Ferramenta 2: `get_patient_by_id`**
* **Descrição:** Recupera o registro completo de um paciente específico a partir do seu ID numérico único.
* **Quando Usar:** Utilize esta função **apenas** quando o usuário fornecer explicitamente um número que se pareça com um ID de paciente. Por exemplo: "Qual o prontuário do paciente 12345?", "Verifique os dados do ID 98765".
* **Argumentos:**
    * `patient_id` (inteiro): O ID numérico do paciente, extraído diretamente da solicitação.

# **REGRAS E DIRETRIZES**
1.  **Prioridade ao ID:** Se um ID numérico de paciente for mencionado, a ferramenta `get_patient_by_id` deve ser sempre a escolhida.
2.  **Não Responda:** Nunca, sob nenhuma circunstância, gere uma resposta em linguagem natural para a pergunta do usuário. Sua saída deve ser **exclusivamente** o objeto JSON.
3.  **Escolha Única:** Selecione apenas uma ferramenta por vez.
4.  **Inferência Mínima:** Baseie sua decisão estritamente nas informações fornecidas no input. Não invente IDs ou tópicos.

# **FORMATO DA RESPOSTA FINAL (OBRIGATÓRIO)**
Sua saída deve ser um JSON válido. Sem exceções.

* **Caso 1: Usar `get_patient_by_id`**
    {
      "tool_call": {
        "name": "get_patient_by_id",
        "arguments": {
          "patient_id": <id_extraido_do_input>
        }
      }
    }

* **Caso 2: Usar `get_similar_documents`**
    {
      "tool_call": {
        "name": "get_similar_documents",
        "arguments": {
          "main_topic": "<topico_principal_extraido>"
        }
      }
    }

* **Caso 3: Nenhuma ferramenta é adequada** (Ex: "Olá, tudo bem?", "Qual o seu nome?")
    
    {
      "tool_call": null
    }

# **EXEMPLOS PRÁTICOS**

**Exemplo 1:**
* **user_input:** "Preciso do histórico médico completo do paciente 84502."
* **Sua Resposta:**
    {
      "tool_call": {
        "name": "get_patient_by_id",
        "arguments": {
          "patient_id": 84502
        }
      }
    }

**Exemplo 2:**
* **user_input:** "Me explique quais são os principais sintomas de um ataque cardíaco."
* **Sua Resposta:**
    {
      "tool_call": {
        "name": "get_similar_documents",
        "arguments": {
          "main_topic": "sintomas de ataque cardíaco"
        }
      }
    }

**Exemplo 3:**
* **user_input:** "Obrigado pela ajuda!"
* **Sua Resposta:**
    
    {
      "tool_call": null
    }
    """

In [29]:
client = Client(host="http://plnmed.incor.usp.br:5434/")
model_name = "gpt-oss:20b"

mysql_connection = sqlite3.connect('pacientes.db')
cursor = mysql_connection.cursor()
cursor.execute("SELECT * FROM pacientes")
patient_id = cursor.fetchone()[0]

user_input = f"O paciete com i {patient_id} possui alguma doenca?"
user_input = f"Quais as principais causas de morte de doenças cardiacas"

response = client.chat(
    model=model_name,
    messages=[
        {"role": "system", "content": router_system_prompt},
        {"role": "user", "content": user_input}
    ]
)

router_agent_response = json.loads(response.message.content)

In [30]:
router_agent_response

{'tool_call': {'name': 'get_similar_documents',
  'arguments': {'main_topic': 'causas de morte em doenças cardíacas'}}}

In [31]:
retrieved_data = {
    'data': None,
    'used_tool': None
}

if router_agent_response['tool_call'] is None:
  retrieved_data['data'] = "O agente julgou que não é necessário utilizar nenhuma ferramenta para responder a pergunta"

else:
  if router_agent_response['tool_call']['name'] == "get_similar_documents":
    main_topic = router_agent_response['tool_call']['arguments']['main_topic']
    documents = get_similar_documents(query_text=main_topic, n_documents=10)
    retrieved_data['data'] = documents
    retrieved_data['used_tool'] = router_agent_response['tool_call']

    if documents is None:
      retrieved_data['data'] = "Não existe documentos similares relacionados ao tópico."

  if router_agent_response['tool_call']['name'] == "get_patient_by_id":
    patient_id = router_agent_response['tool_call']['arguments']['patient_id']
    patient_data = get_patient_by_id(patient_id)
    retrieved_data['data'] = patient_data
    retrieved_data['used_tool'] = router_agent_response['tool_call']

    if patient_data is None:
      retrieved_data['data'] = "O id do paciente é inexistente ou está incorreto."

In [32]:
retrieved_data

{'data': {'documents': ['O VO2pico obtido através da equação preditora elaborada \nneste estudo com base no desempenho no TUG demonstrou \nconcordância com o VO2pico determinado pelo TCPE na \nmesma amostra, demonstrando ser um método adequado \npara estimar a ACR de cardiopatas. Em uma metanálise \ncom adultos saudáveis, Kodama et al.12 sugeriram que a \nACR seria um importante preditor de mortalidade e eventos \ncardiovasculares. Apesar da sua amostra apresentar diferentes \ncaracterísticas do nosso estudo, é possível inferir que uma \nmelhor ACR está associada a menores riscos de complicações \ncardiovasculares. É importante frisar que na literatura ainda são \nescassos os estudos que relacionem o teste TUG à população \nde pacientes cardiopatas A ACR estabelecida pelo VO2pico é um importante \ncomponente de avaliação da saúde, pois, de acordo com \nCarvalho et al.3 e Ritt et al.,5 trata-se de um determinante \nque deve ser mensurado periodicamente em pacientes \ncardiopatas, com o 

##### Tarefa 3: Criar agente resposável por gerar a resposta final para o usuário

In [33]:
# Passo 1
## Criar um prompt do agente responsável pelo fornecimento da resposta com base na entrada

In [34]:
history = []

In [35]:
final_response_agent_prompt = f"""
## **HISTÓRICO DA CONVERSA**
{history}
## **DADOS DE CONTEXTO**
{retrieved_data}

=============================================================

## **OBJETIVO**
Responda objetivamente a pergunta.
Você receberá dados de contexto e seu dever será utilizá-lo e com base nos dados de contexto e no seu conhecimento você deverá responder a seguinte pergunta do usuário:
"""

In [36]:
response_final_agent = client.chat(
    model=model_name,
    messages=[
        {"role": "system", "content": final_response_agent_prompt},
        {"role": "user", "content": user_input}
    ]
)

final_response = response_final_agent.message.content

history_input = {
    "user_input": user_input,
    "agent_response": final_response
}

history.append(history_input)

In [37]:
display(Markdown(final_response))

**Principais causas de morte em doenças cardíacas (cardiopatia)**  

| Classificação | Descrição | Por que é frequente |
|---------------|-----------|---------------------|
| **Infarto agudo do miocárdio (IAM)** | Ocorre quando há obstrução súbita de uma artéria coronária, levando à necrose do miocárdio. | A principal manifestação de doença arterial coronariana (DAC) e a maior causa de mortalidade cardiovascular em todo o mundo. |
| **Insuficiência cardíaca (IC)** | Deterioração progressiva da função ventricular que leva a sintomas de congestão e descompensação. | A evolução crônica de DAC, hipertensão, miocardiopatias e outras doenças cardíacas pode culminar em morte por descompensação ou complicações secundárias. |
| **Arritmias ventriculares (taquicardia/ fibrilação ventricular)** | Ritmos elétricos anormais que podem levar à parada cardíaca súbita. | São a principal causa de morte súbita cardíaca, especialmente em pacientes com doença coronariana ou miocardiopatias. |
| **Estopelo ou embolia coronária** | Bloqueio de uma artéria coronária por trombo, causando morte localizada ou disseminada. | A trombose de ateroma é a causa mais comum de IAM, podendo evoluir para morte. |
| **Valvulopatias severas (por exemplo, estenose aórtica)** | Dano mecânico e hemodinâmico que pode levar a insuficiência cardíaca ou arritmias. | Em pacientes idosos, a doença valvular pode ser a principal causa de morte cardiovascular. |
| **Complicações de doenças cardíacas crônicas** | Infecções (endocardite), aneurismas, pericardite, etc. | Em ambientes com recursos limitados, a falta de monitoramento adequado pode levar a essas complicações fatais. |

**Resumo**  
A maioria das mortes associadas a doenças cardíacas tem origem na doença arterial coronariana – tanto em forma aguda (IAM) quanto em forma crônica (insuficiência cardíaca). Arritmias ventriculares e a morte súbita cardíaca são as principais causas de falecimento súbito, enquanto complicações valvulares e de doenças cardíacas crônicas também contribuem significativamente, especialmente em populações com acesso limitado a exames de diagnóstico e tratamento especializado.